In [2]:
# !pip install nltk

In [3]:
# 데이터 준비

import nltk
from nltk.corpus import movie_reviews

# nltk.download("movie_reviews")
# print( movie_reviews.fileids() )
reviews = [ movie_reviews.raw(fid) for fid in movie_reviews.fileids() ]
categories = [ movie_reviews.categories(fid) for fid in movie_reviews.fileids() ]

In [4]:
import pandas as pd
print( type(reviews), len(reviews) )
pd.DataFrame(categories).value_counts()

<class 'list'> 2000


0  
neg    1000
pos    1000
Name: count, dtype: int64

In [5]:
# 데이터 준비 2
import numpy as np
from sklearn.preprocessing import LabelEncoder

labeled_categories = LabelEncoder().fit_transform(categories).tolist()
np.unique(labeled_categories, return_counts=True)

c:\Users\human\.conda\envs\human-dl-env2\lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(array([0, 1]), array([1000, 1000], dtype=int64))

In [6]:
# 데이터 준비 3
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(reviews, labeled_categories, random_state=42)

In [7]:
# 사전 학습 모델 준비 1

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_inputs = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")   # "tf"
test_inputs = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")     # "tf"

In [8]:
train_inputs

{'input_ids': tensor([[ 101, 2182, 1005,  ..., 4110, 2011,  102],
        [ 101, 1996, 2236,  ..., 1012, 2002,  102],
        [ 101, 4626, 2520,  ..., 1996, 4637,  102],
        ...,
        [ 101, 1045, 3984,  ..., 2009, 2453,  102],
        [ 101, 1996, 3614,  ...,    0,    0,    0],
        [ 101, 1045, 2031,  ..., 8197, 4892,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [31]:
import torch

class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }
        input["label"] = self.labels[idx]
        return input

In [10]:
train_dataset = ReviewDataset(train_inputs, y_train)
test_dataset = ReviewDataset(test_inputs, y_test)

In [100]:
for input in train_dataset:
    print(input.keys())
    # print(input['input_ids'])


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])


C:\Users\human\AppData\Local\Temp\ipykernel_11760\1308800813.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = { k: torch.tensor(v) for k, v in self.inputs.items() }


In [12]:
device_name = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
print(device)

cpu


In [13]:
# !pip install evaluate

In [15]:
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

def compute_metrics(evaluation_dataset):
    logits, labels = evaluation_dataset
    predictions = np.argmax(logits, axis=-1)
    # return metric.compute(predictions=predictions, references=labels)
    return { "accuracy": (predictions==labels).sum() / len(labels) }

train_args = TrainingArguments(
    output_dir="checkpoints",
    num_train_epochs=10,
    eval_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8
)

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    args=train_args
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()